In [67]:
# Import packages
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import platform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

sc.logging.print_header()
print(platform.system(), platform.release())
print("python", platform.python_version())

scanpy==1.10.3 anndata==0.10.9 umap==0.5.7 numpy==1.26.4 scipy==1.13.1 pandas==2.2.3 scikit-learn==1.1.2 statsmodels==0.14.4 pynndescent==0.5.13
Linux 5.15.167.4-microsoft-standard-WSL2
python 3.9.21


In [14]:
# Load data
hdf5_file_path = 'data/caner_mutant_peptides.h5'
df = pd.read_hdf(hdf5_file_path, key='dataset_name')

# Check the data structure
print(df.shape)
df['response_type'].value_counts()

(1787710, 57)


response_type
not_tested    1364625
negative       422907
CD8               178
Name: count, dtype: int64

In [33]:
# Define the predictors
with open('predictors.txt', 'r') as f:
    predictors = [line.strip() for line in f]

len(predictors)

32

In [52]:
# Select relevant columns (response_type and predictors)
selected_columns = ['response_type', 'mutant_seq'] + predictors
df_selected = df[selected_columns]

# Filter out rows where 'response_type' is 'not_tested' for training data
df_filtered = df_selected[df_selected['response_type'] != 'not_tested']
# Update the categories of 'response_type'
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['response_type'] = df_filtered['response_type'].cat.remove_categories(['not_tested'])

# Check the types of the target
df_filtered['response_type'].value_counts()

response_type
negative    422907
CD8            178
Name: count, dtype: int64

In [ ]:
# Save the rows with 'not_tested' for later exploration
df_not_tested = df_selected[df_selected['response_type'] == 'not_tested']
df_not_tested.to_hdf('data/df_not_tested.h5', key='not_tested', mode='w', format="table")

In [53]:
# Impute missing values for numeric columns with the mean
numeric_columns = df_filtered.select_dtypes(include=['float64', 'int64']).columns
df_filtered[numeric_columns] = df_filtered[numeric_columns].fillna(df_filtered[numeric_columns].mean())

# Impute missing values for categorical predictors with 'missing'
categorical_columns = df_filtered.select_dtypes(include=['category', 'object']).columns
for col in categorical_columns:
    if df_filtered[col].dtype.name == 'category':
        df_filtered[col] = df_filtered[col].cat.add_categories('missing')
    df_filtered[col] = df_filtered[col].fillna('missing')

# Ensure that 'response_type' doesn't contain the 'missing' category
if df_filtered['response_type'].dtype.name == 'category':
    df_filtered['response_type'] = df_filtered['response_type'].cat.remove_categories('missing')

# Reset the index only if necessary, e.g., after row removal
# df_filtered = df_filtered.reset_index(drop=True)

# Double-check the distribution of the target variable 'response_type'
print(df_filtered['response_type'].value_counts())

response_type
negative    422907
CD8            178
Name: count, dtype: int64


In [54]:
# Prepare data for ML

# Separate the 'mutant_seq' column
mutant_seq_column = df_filtered['mutant_seq']

# Drop 'mutant_seq' and 'response_type' columns from the predictors for model training
X = df_filtered.drop(columns=['mutant_seq', 'response_type'])
y = df_filtered['response_type']

# Encode the target variable as binary
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Converts 'yes'/'no' to 1/0

# Convert columns with non-numeric types to category dtype
categorical_columns = ['bestWTMatchType_I', 'Zygosity', 'Clonality', 
                       'mutation_driver_statement_Intogen', 'gene_driver_Intogen']

# Ensure the columns are of type 'category'
for col in categorical_columns:
    df_filtered[col] = df_filtered[col].astype('category')

# 2. Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# 4. Define preprocessors
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Standardize numeric features
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# 5. Combine preprocessors in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 6. Apply preprocessing to the training set and test set
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

#
X_train_processed.shape, X_test_processed.shape

((338468, 26), (84617, 26))

In [55]:
# Try to only predict "negative" (CD8) with high purity.
    
model = xgb.XGBClassifier(
    objective="binary:logistic", 
    random_state=42,
    scale_pos_weight=1.5,  # Adjust the weight for optimization
    eval_metric="logloss",
    colsample_bytree=0.4,
)

model.fit(X_train_processed, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test_processed)
y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

# Prediction metrics

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print results
print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
# print(confusion_matrix(y_test, y_pred_adjusted))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9996454613139204
ROC AUC Score: 0.5606001493226028
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.12      0.21        33
           1       1.00      1.00      1.00     84584

    accuracy                           1.00     84617
   macro avg       0.90      0.56      0.61     84617
weighted avg       1.00      1.00      1.00     84617

Confusion Matrix:
[[    4    29]
 [    1 84583]]


In [64]:
# Extract the predicted neoantigen sequence for wet validation
CD8_pred_index = np.where(y_pred == 0)[0]  # Indices where predictions are 0

mutant_seq_pred = df_filtered.iloc[X_test.index[CD8_pred_index]]

# 9. Optionally, print or inspect the results
print(mutant_seq_pred)

       response_type   mutant_seq  mutant_rank  mutant_rank_netMHCpan  \
248746      negative   GADGVGKSAL         0.08                  0.171   
405655           CD8  YPAAVNTIVAI         0.80                  0.397   
268852           CD8    TADFDITEL         0.06                  0.116   
248744           CD8    GADGVGKSA         0.06                  1.307   
320008           CD8    FVVPYMIYL         0.03                  0.123   

        mutant_rank_PRIME  mut_Rank_Stab  TAP_score  mut_netchop_score_ct  \
248746               0.30          15.00      0.568              0.438231   
405655               0.20           0.03     -0.160              0.940328   
268852               0.05          19.00      0.648              0.974604   
248744               0.30          25.00     -0.922              0.102754   
320008               0.01           3.00      0.976              0.975419   

        mut_binding_score  mut_is_binding_pos  ...  bestWTPeptideCount_I  \
248746           3.754

In [65]:
# Alternative strategy: Try to increase ture "negative" but decrease false "negative"
# Increase the chance to identify more neoantigen hits but also more false "negative". 

# SMOTE option
smote = SMOTE(sampling_strategy=0.2, random_state=42)  # Increase synthetic samples for class 0
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

# Train the XGBoost model with the resampled data
model = xgb.XGBClassifier(
    objective="binary:logistic",
    random_state=42,
    scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train), 
    eval_metric="logloss",
    use_label_encoder=False,
    #max_leaves=60
)

model.fit(X_train_resampled, y_train_resampled)

# Evaluate the model on the test set
y_pred = model.predict(X_test_processed)
y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

#
threshold = 0.002  # Lower the threshold to favor predicting class 0
y_pred_adjusted = (y_pred_proba > threshold).astype(int)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred_adjusted)
roc_auc = roc_auc_score(y_test, y_pred_adjusted)

# Print results
print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_adjusted))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_adjusted))

Accuracy: 0.991691976789534
ROC AUC Score: 0.814097121312434
Classification Report:
              precision    recall  f1-score   support

           0       0.03      0.64      0.06        33
           1       1.00      0.99      1.00     84584

    accuracy                           0.99     84617
   macro avg       0.51      0.81      0.53     84617
weighted avg       1.00      0.99      1.00     84617

Confusion Matrix:
[[   21    12]
 [  691 83893]]


In [68]:
# Random forest algorithm
# The outcome seems similar to the above. 

# Apply SMOTE to balance the class distribution in the training set
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

# Train the Random Forest model with the resampled data
model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    n_estimators=100,
    max_depth=10,
    n_jobs=-1
)
model.fit(X_train_resampled, y_train_resampled)

# Evaluate the model
y_pred = model.predict(X_test_processed)
y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print results
print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

Accuracy: 0.9866102556223927
ROC AUC Score: 0.8418464771616669
              precision    recall  f1-score   support

           0       0.02      0.70      0.04        33
           1       1.00      0.99      0.99     84584

    accuracy                           0.99     84617
   macro avg       0.51      0.84      0.52     84617
weighted avg       1.00      0.99      0.99     84617

[[   23    10]
 [ 1123 83461]]


In [ ]:
# h2o platform (time consuming to run locally!)
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O
h2o.init()

# Convert the data to H2OFrame for AutoH2O
train_data = pd.concat([X_train, y_train], axis=1)
train_data_h2o = h2o.H2OFrame(train_data)

# Define target and features
target = 'response_type'
features = X_train.columns.tolist()

# Train the AutoML model
aml = H2OAutoML(max_models=20, seed=1, balance_classes=True)
aml.train(x=features, y=target, training_frame=train_data_h2o)

# View leaderboard and get the best model
aml.leaderboard
best_model = aml.leader

# Make predictions on the test set
test_data_h2o = h2o.H2OFrame(X_test)
predictions = best_model.predict(test_data_h2o)

# View predictions (convert to dataframe)
predictions_df = predictions.as_data_frame()
print(predictions_df.head())

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.5" 2024-10-15; OpenJDK Runtime Environment (build 21.0.5+11-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 21.0.5+11-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /home/yc/miniconda3/envs/py39/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpcen_gi76
  JVM stdout: /tmp/tmpcen_gi76/h2o_yc_started_from_python.out
  JVM stderr: /tmp/tmpcen_gi76/h2o_yc_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


In [98]:
# Focal loss to deal with imbalanced data
# Hyperparameter tuning  required could be tedious for optimization. 

# 1. Define Focal Loss for Binary Classification with Gradient and Hessian
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_inner(y_true, y_pred):
        # Clip values to prevent log(0)
        epsilon = 1e-8
        y_true = np.clip(y_true, epsilon, 1 - epsilon)
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

        # Cross-entropy loss
        log_loss = -y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred)
        
        # Focal loss component
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_loss = alpha * (1 - p_t) ** gamma * log_loss

        # Compute gradient and hessian
        grad = alpha * (1 - p_t) ** gamma * (y_pred - y_true) / (y_pred * (1 - y_pred))
        hess = alpha * (1 - p_t) ** gamma * (1 - 2 * y_pred) / (y_pred * (1 - y_pred))

        return grad, hess

    return focal_loss_inner

# 2. Calculate scale_pos_weight for Class Imbalance
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# 3. Initialize XGBoost model with focal loss as the custom objective function
model = xgb.XGBClassifier(
    objective=focal_loss(gamma=2.0, alpha=0.25),  # Use custom focal loss
    random_state=42,
    scale_pos_weight=scale_pos_weight,  # Handle class imbalance
    learning_rate=0.05,  # Slower learning rate to focus more on both classes
    n_estimators=500,  # Increase number of trees to make up for slow learning
    max_depth=6,  # Shallower trees to avoid overfitting
    min_child_weight=2,  # Conservative splitting
    subsample=0.8,  # Prevent overfitting by subsampling rows
    colsample_bytree=0.8,  # Subsampling features
    gamma=0.1,  # Control splits with small regularization
    booster="dart",  # Use dropout trees to improve generalization
    eval_metric="auc"  # Focus on AUC for class imbalance
)

# 4. Train the model on the original training data (without SMOTE)
model.fit(X_train_processed, y_train)

# 5. Make predictions on the test data
y_pred = model.predict(X_test_processed)
y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

# 6. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print the results
print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")
print(classification_report(y_test, y_pred))

# 7. Optional: Adjust the threshold for the positive class
# Lowering the threshold increases recall for the minority class
threshold = 0.3
y_pred_adjusted = (y_pred_proba > threshold).astype(int)

# 8. Evaluate with the adjusted threshold
print("Evaluation with adjusted threshold:")
print(classification_report(y_test, y_pred_adjusted))

Accuracy: 0.9996100074453124
ROC AUC Score: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        33
           1       1.00      1.00      1.00     84584

    accuracy                           1.00     84617
   macro avg       0.50      0.50      0.50     84617
weighted avg       1.00      1.00      1.00     84617

Evaluation with adjusted threshold:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        33
           1       1.00      1.00      1.00     84584

    accuracy                           1.00     84617
   macro avg       0.50      0.50      0.50     84617
weighted avg       1.00      1.00      1.00     84617



/home/yc/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yc/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yc/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yc/m

In [ ]:
# lce (Good idea but yet to be validated)
# ValueError to fix: Cannot cast object dtype to float64.
# Time consuming.

# Select all categorical columns in the dataset
categorical_columns = X_train.select_dtypes(include=['category', 'object']).columns

# Perform one-hot encoding for categorical features
X_train_encoded = pd.get_dummies(X_train, columns=categorical_columns)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_columns)

# Ensure all data is numeric (after one-hot encoding, it should be)
X_train_encoded = X_train_encoded.apply(pd.to_numeric, errors='coerce')
X_test_encoded = X_test_encoded.apply(pd.to_numeric, errors='coerce')

# Fit the model
from lce import LCEClassifier
clf = LCEClassifier(n_jobs=-1, random_state=0)
clf.fit(X_train_encoded, y_train)

# Make predictions and compute accuracy
y_pred_lce = clf.predict(X_test_encoded)
accuracy = accuracy_score(y_test, y_pred_lce)
print("Accuracy: {:.1f}%".format(accuracy * 100))
print(classification_report(y_test, y_pred_lce))